# Plastic glacier profiles on idealized bed

In [ ]:
import chakra
import chakra_sermeq
import matplotlib.pyplot as plt
import numpy as np
from oggm import cfg

### Set up standard idealized bed profile

In [ ]:
# Bassis & Ultee bed profile
fls = chakra.bu_tidewater_bed()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
x = fls[-1].dis_on_line * fls[-1].dx_meter / 1000
ax.hlines(0, 0, 60, color='C0')
ax.plot(x, fls[-1].bed_h, color='k')
ax.set(ylim=(-350, 800), xlabel=('Distance along flowline [km]'), ylabel=('Altitude [m]'));
plt.show()

In [ ]:
len(fls[-1].bed_h)

### Produce a SERMeQ-plastic surface profile

For now, we are going to call a special class `chakra_sermeq.PlasticGlacier`.  Its function `plastic_profile` will produce a basic analytical surface profile for a quasi-steady-state plastic calving glacier.  This function was superseded by some fancier functionality in later SERMeQ, and we'll probably have to rewrite it again.  But it should do the job to start.

In [ ]:
g = chakra_sermeq.PlasticGlacier(yield_strength=50e3)
x_nondim = x/(chakra_sermeq.L0/1000) # x is in km, not m
bed_nondim = fls[-1].bed_h/chakra_sermeq.H0
g.set_bed_function(x_nondim, bed_nondim)

init_point = 40000/chakra_sermeq.L0 # initialize 40 km down flowline
s = g.plastic_profile(Bfunction=g.bingham_const,
                  startpoint=init_point, endpoint=min(x_nondim),
                  hinit=g.balance_thickness(g.bed_function(init_point), g.bingham_const())
                     )

We need to translate from nondimensional to dimensional coordinates to display on the same plot as OGGM.  Notice that we are only storing x-coordinates (and bed and surface) where there is nonzero ice thickness--the model simply quits once it reaches the terminus.

In [ ]:
xs_dimensional = 10*np.array(s[0]) # multiply by 10, or L0/1000, because original units km
bs_dimensional = chakra_sermeq.H0*np.array(s[2])
ss_dimensional = chakra_sermeq.H0*np.array(s[1])

In [ ]:
max(xs_dimensional)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
ax.plot(xs_dimensional, bs_dimensional, color='k') # bed topography
ax.plot(xs_dimensional, ss_dimensional, color='b') # glacier surface
ax.plot(x, fls[-1].bed_h, color='k', ls='-.') # topo from ideal OGGM
ax.set(ylim=(-350, 800), xlabel=('Distance along flowline [km]'), ylabel=('Altitude [m]'), aspect=0.01);

### Produce a plastic profile initialized by left-boundary flux

Convert an influx in m3/a to a flux-balancing thickness at the upstream boundary.  For now I use an idealized, mean Glen's-law velocity for a laterally confined glacier. Down the road we probably want to enforce continuity between OGGM and SERMeQ (OGGM passes velocity to SERMeQ along with flux).

In [ ]:
width = 600 #m
fx = 0.6 * cfg.SEC_IN_YEAR # m3/a
# u_in = 100 #m/a, ice velocity
u_in = chakra_sermeq.glen_u(width, basal_yield=210e3) * cfg.SEC_IN_YEAR # m/a
flux_balance_thickness = fx/(u_in*width)

s_fx = g.plastic_profile(Bfunction=g.bingham_const,
                  startpoint=0, endpoint=max(x_nondim),
                  hinit=flux_balance_thickness/chakra_sermeq.H0
                     )

xsx_dimensional = 10*np.array(s_fx[0]) # multiply by 10, or L0/1000, because original units km
bsx_dimensional = chakra_sermeq.H0*np.array(s_fx[2])
ssx_dimensional = chakra_sermeq.H0*np.array(s_fx[1])

In [ ]:
print(u_in)
print(flux_balance_thickness)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
ax.plot(xsx_dimensional, bsx_dimensional, color='k') # bed topography
ax.plot(xsx_dimensional, ssx_dimensional, color='b') # glacier surface
ax.plot(x, fls[-1].bed_h, color='k', ls='-.') # topo from ideal OGGM
ax.set(ylim=(-350, 2000), xlabel=('Distance along flowline [km]'), ylabel=('Altitude [m]'), aspect=0.01);

There are pretty tight combinations of flux and velocity (basal yield strength, Glen A) that let this work.  (f, 10f) gets the glacier snout down 36.8 km, but note what happens when we reduce flux by 10% e.g. (f0, u_in)->(9, 100) versus (10, 100).